# Scikit-HEP ecosystem updates

**Note:** although I'm presenting in Jupyter, this is a talk, rather than a tutorial. You don't have to follow along.

<br><br><br><br><br>

## State of the ecosystem

<table width="100%"><tr style="background: white">
    <td align="left" width="50%"><img src="img/shells-border.png" width="95%"></td>
    <td align="right" width="50%"><img src="img/shells-hep.png" width="95%"></td>
</tr></table>

<br><br><br><br><br>

<img src="img/pip-allos-scikithep-log.png" width="100%">

<br><br><br><br><br>

<img src="img/pip-macwin-scikithep-log.png" width="100%">

<br><br><br><br><br>

## There are more tools than I could reasonably tell you about

And that's good! Ideally,

   * each tool does one thing well
   * is maintained by enthusiastic developers with recognition for their work
   * who also know about each other and can ensure that their tools work together

<br><br><br><br><br>

## Illustrative vertical slice: Uproot → Awkward Array → Vector → fastjet → hist

Why these five?

<br><br>

<img src="img/uproot-logo.png" width="200px">

<p style="font-size: 14pt">Reads ROOT data as <span style="background: yellow">arrays</span>.</p>

<br><br>

<img src="img/awkward-logo.png" width="200px">

<p style="font-size: 14pt">Manipulates <span style="background: yellow">arrays</span> of arbitrary data structures.</p>

<br><br>

<img src="img/vector-logo.png" width="200px">

<p style="font-size: 14pt">Manipulates <span style="background: yellow">arrays</span> of 2D, 3D, and Lorentz vectors.</p>

<br><br>

<img src="img/fastjet-logo-300px.png" width="200px">

<p style="font-size: 14pt">Finds jets in <span style="background: yellow">arrays</span> of Lorentz vectors.</p>

<br><br>

<img src="img/hist-logo.png" width="200px">

<p style="font-size: 14pt">Fills histograms with <span style="background: yellow">arrays</span>.</p>

<br><br><br><br><br>

## Major trend (back) to talking about arrays

<img src="img/chep-papers-paradigm.png" width="85%">

<br><br><br><br><br>

## Speedrun through the vertical slice

Get a TTree with Uproot (from a tutorial 2 years ago).

In [ ]:
import uproot

In [ ]:
tree = uproot.open("https://github.com/jpivarski-talks/2020-04-08-eic-jlab/raw/master/open_charm_18x275_10k.root:events/tree")
tree

Read some TBranches from it.

In [ ]:
components = tree.arrays(["px", "py", "pz", "tot_e"])
components

Reformat them into an array of lists of four-vectors.

In [ ]:
import awkward as ak
import vector
vector.register_awkward()

In [ ]:
events = ak.zip(
    {"px": components.px, "py": components.py, "pz": components.pz, "E": components.tot_e},
    with_name="Momentum4D",
)
events

See that each list has a different length.

In [ ]:
ak.num(events)

Run FastJet's anti-$k_T$ clustering algorithm on all events.

In [ ]:
import fastjet

In [ ]:
cluster_sequence = fastjet.ClusterSequence(
    events,
    fastjet.JetDefinition(fastjet.antikt_algorithm, 0.5),
)
cluster_sequence

In [ ]:
clustered_events = cluster_sequence.inclusive_jets()
clustered_events

Histogram the number of particles in events and the number of jets in events.

In [ ]:
import hist

In [ ]:
hist.new.Reg(101, -0.5, 100.5).Double().fill(ak.num(events))

In [ ]:
hist.new.Reg(101, -0.5, 100.5).Double().fill(ak.num(clustered_events))

See what happens when you change $\Delta R$!

<br><br><br><br><br>

## Principle 1: communication via arrays

Scientific Python beyond HEP communicates via arrays; mostly, but not exclusively NumPy.

[NumPy](https://numpy.org/), [Pandas](https://pandas.pydata.org/), [CuPy](https://cupy.dev/), [Sparse](https://sparse.pydata.org/), [xarray](https://docs.xarray.dev/), [Pint](https://pint.readthedocs.io/), [uncertainties](https://pythonhosted.org/uncertainties/numpy_guide.html), [Dask](https://dask.org/), [JAX](https://jax.readthedocs.io/), [TensorFlow](https://www.tensorflow.org/), [PyTorch](https://pytorch.org/), ... can often be interchanged (duck type).

In [ ]:
import graphviz

In [ ]:
# Credit: Stephan Hoyer https://gist.github.com/shoyer/7936a93843354299368145e8266c5d83
# libraries that only interact with NumPy are not listed, e.g., TensorFlow, PyTorch, scipy.sparse
g = graphviz.Digraph()
g.edge("Dask", "NumPy")
g.edge("Dask", "CuPy")
g.edge("Dask", "Sparse")
g.edge("Dask", "NumPy MaskedArray")
g.edge("Dask", "Pandas")
g.edge("CuPy", "NumPy")
g.edge("Sparse", "NumPy")
g.edge("NumPy MaskedArray", "NumPy")
g.edge("Pandas", "NumPy")
g.edge("Pandas", "NumPy MaskedArray")
g.edge("JAX", "NumPy")
g.edge("Pint", "Dask")
g.edge("Pint", "Pandas")
g.edge("Pint", "NumPy")
g.edge("xarray", "Dask")
g.edge("xarray", "CuPy")
g.edge("xarray", "Sparse")
g.edge("xarray", "NumPy")
g.edge("xarray", "Pandas")
g.edge("xarray", "NumPy MaskedArray")
g.edge("xarray", "Pint")
g.edge("xarray", "JAX")
g

<br><br><br><br><br>

Awkward Array is our contribution to that list.

In [ ]:
import numpy as np

In [ ]:
import skhep_testdata

In [ ]:
array = uproot.open(skhep_testdata.data_path("uproot-HZZ.root"))["events"].arrays()
array

In [ ]:
array[0]

In [ ]:
nparray = ak.to_numpy(array[["MET_px", "MET_py"]])
nparray

<br><br><br><br><br>

But so are Vector's array backends.

In [ ]:
vector.MomentumNumpy2D.mro()

In [ ]:
nparray.dtype.names

In [ ]:
nparray.dtype.names = ["px", "py"]

In [ ]:
npvectors = nparray.view(vector.MomentumNumpy2D)
npvectors

In [ ]:
npvectors.pt

In [ ]:
npvectors.to_rhophi()

In [ ]:
abs(npvectors)

In [ ]:
npvectors + vector.obj(pt=100, phi=np.pi)

<br><br><br><br><br>

Vector (currently!) has three backends:

   * Python object of one vector (for completeness)
   * NumPy subclass of many vectors
   * Awkward Array of vectors in lists and other structures

To make an Awkward Array of Lorentz vectors, name the record `"Momentum4D"` (using `with_name`), name the fields vector components (using `ak.zip`), and

```python
vector.register_awkward()
```

once globally.

Here's a slick one-liner:

In [ ]:
akvectors = ak.zip(dict(zip(["px", "py", "pz", "E"], ak.unzip(array[["Muon_Px", "Muon_Py", "Muon_Pz", "Muon_E"]]))), with_name="Momentum4D")
akvectors

In [ ]:
akvectors.pt

In [ ]:
akvectors.to_rhophietat()

In [ ]:
abs(akvectors)

In [ ]:
akvectors + vector.obj(pt=100, phi=np.pi, eta=0.5, E=100)

<br><br><br><br><br>

Quick example: $\Delta R$ between all pairs of muons.

In [ ]:
mu1, mu2 = ak.unzip(ak.combinations(akvectors, 2))
mu1, mu2

In [ ]:
mu1.deltaR(mu2)

<br><br><br><br><br>

fastjet communicates via arrays, too. These are Awkward Arrays of Vectors.

In [ ]:
events

In [ ]:
cluster_sequence = fastjet.ClusterSequence(
    events,
    fastjet.JetDefinition(fastjet.antikt_algorithm, 0.5),
)
cluster_sequence

In [ ]:
cluster_sequence.inclusive_jets()

<br><br><br><br><br>

The same library provides the classic FastJet Python interface, in which each vector is a `fastjet.PseudoJet`.

In [ ]:
inclusive_jets = []
for event in events:
    pseudojets = []
    for particle in event:
        pseudojets.append(fastjet.PseudoJet(particle.px, particle.py, particle.pz, particle.E))

    cluster_sequence = fastjet.ClusterSequence(
        pseudojets,
        fastjet.JetDefinition(fastjet.antikt_algorithm, 0.5),
    )
    inclusive_jets.append(cluster_sequence.inclusive_jets())

In [ ]:
inclusive_jets[0]

In [ ]:
print("\n".join(f"<PseudoJet px={obj.px():.4g}, py={obj.py():.4g}, pz={obj.pz():.4g}, E={obj.E():.4g}>" for obj in inclusive_jets[0]))

<br><br><br><br><br>

## Principle 2: loose coupling

These interfaces demand the minimum of what they need.

If another library provides too much information, it's not a problem.

In [ ]:
charge = tree["charge"].array()
charge

In [ ]:
events = ak.zip(
    {
        "px": components.px, "py": components.py, "pz": components.pz, "E": components.tot_e,
        "charge": charge,
    },
    with_name="Momentum4D",
)
events

In [ ]:
print(events.type)

<br><br><br><br><br>

Vectors don't need a `charge` field, and it's ignored in vector operations.

In [ ]:
abs(events)

In [ ]:
p1, p2 = ak.unzip(ak.combinations(events, 2))
p1, p2

In [ ]:
p1 + p2

In [ ]:
print((p1 + p2).type)

<br><br><br><br><br>

Jet-finding doesn't need a `charge` field, either.

In [ ]:
cluster_sequence = fastjet.ClusterSequence(
    events,
    fastjet.JetDefinition(fastjet.antikt_algorithm, 0.5),
)
cluster_sequence

In [ ]:
cluster_sequence.inclusive_jets()

In [ ]:
print(cluster_sequence.inclusive_jets().type)

<br><br><br><br><br>

But it's passed through wherever possible.

In [ ]:
cluster_sequence.constituents()

In [ ]:
print(cluster_sequence.constituents().type)

In [ ]:
cluster_sequence.constituents().charge

In [ ]:
jet_charge = ak.sum(cluster_sequence.constituents().charge, axis=-1)
jet_charge

In [ ]:
jets = cluster_sequence.inclusive_jets()
jets["charge"] = jet_charge
jets

In [ ]:
print(jets.type)

<br><br><br><br><br>

## Principle 3: array-oriented _or_ Numba-JIT

The major benefit of array-oriented/columnar code is its **interactivity**. The speed is nice, too, but that's a crutch for Python. (See Julia.)

You see results one _operation_ at a time, not one _event_ at a time.

But some algorithms are easier to express in for-loop code than array-oriented code. Don't struggle! Use Numba to (temporarily) switch to fast for loops.

In [ ]:
import numba as nb

In [ ]:
@nb.jit
def sum_of_vectors(events):
    output = np.empty((len(events), 4))
    
    for index, particles in enumerate(events):
        sum_vector = vector.obj(px=0.0, py=0.0, pz=0.0, E=0.0)
        
        for particle in particles:
            sum_vector = sum_vector + particle
        
        output[index, 0] = sum_vector.pt
        output[index, 1] = sum_vector.phi
        output[index, 2] = sum_vector.z
        output[index, 3] = sum_vector.mass
    
    return output

In [ ]:
output = sum_of_vectors(events)
output

In [ ]:
np_sums = output.view([("pt", "f8"), ("phi", "f8"), ("z", "f8"), ("mass", "f8")]).view(vector.MomentumNumpy4D)
np_sums

<br><br><br><br><br>

The JIT-compilation really matters!

In [ ]:
sum_of_vectors.py_func

In [ ]:
%%timeit -n 1 -r 1

sum_of_vectors.py_func(events[:100])

In [ ]:
%%timeit -n 1 -r 1

sum_of_vectors(events[:100])

In [ ]:
len(events)

<br><br><br><br><br>

Not all libraries are Numba-enabled, but it's a growing list.

   * Awkward Array: iteration and ArrayBuilder
   * Vector: single-object vectors and Awkward Arrays of vectors (NumPy arrays of vectors are [scikit-hep/vector#43](https://github.com/scikit-hep/vector/issues/43))
   * hist: experimental prototypes ([scikit-hep/hist#293](https://github.com/scikit-hep/hist/pull/293))